In [2]:
# autoreload for utils.py edits
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt

# helper library
from utils import (
    register_hooks,
    remove_hooks,
    run_and_cache,
    patch_activations,
    compute_metrics,
    plot_patching_results
)

from aeon.datasets import load_classification

## Loading and preprocessing JapaneseVowels data

In [3]:
X_train, y_train = load_classification("JapaneseVowels", split="train")
X_test, y_test = load_classification("JapaneseVowels", split="test")

# swap axes to (N, seq_len, 1) and convert to float32
def prep(X):
    X_np = X.astype(np.float32)
    return np.swapaxes(X_np, 1, 2)

X_test = torch.tensor(prep(X_test))
y_test = torch.tensor(y_test.astype(np.int64))

print("Test set:", X_test.shape, y_test.shape)

Test set: torch.Size([370, 25, 12]) torch.Size([370])
